In [1]:
!pip install transformers datasets wandb

In [2]:
import torch

torch.cuda.is_available()

True

In [3]:
from datasets import load_dataset

import pandas as pd

df = pd.read_csv('EPITOME_pairs2.csv')
df['text'] = '#Sentence:' + df['seeker_post'] + '#Sentence:' + df['response_post']
df['label'] = df['Rate']

df[['text', 'label']].to_csv('EPITOME_pairs3.csv', index=False)

dataset = load_dataset("csv", data_files="EPITOME_pairs3.csv")

KeyError: 'Rate'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
ds = tokenized_dataset['train'].train_test_split(test_size=0.1, shuffle=True, seed=42)
ds_test = ds['test']
ds_train = ds['train'].train_test_split(test_size=0.15, shuffle=True, seed=42)
ds_val = ds_train['test']
ds_train = ds_train['train']

In [ ]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

In [ ]:
%env WANDB_PROJECT = Empathy_project

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='EPITOME10',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
)

trainer.train()

In [ ]:
'''metrics = trainer.evaluate()
print(metrics)'''